In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("statements_uni.csv")
df.head()

,Source,Targets,Properties
0,1,"36906466,3695190,497745,1133705,1139177,743945...","31,279,398,398,398,398,398,398,398,461,527,527..."
1,2,"3504248,36133,2293672,323,3400,7879772,1858996...","31,138,163,170,170,276,361,397,398,398,398,398..."
2,3,"937228,203872,483247,1322005,7239,4,4,7239,299...","31,279,279,279,361,461,527,527,527,910,1343,13..."
3,4,"2996394,2956046,3505845,3,3,14819852,2383867,2...","31,279,279,361,461,461,527,527,527,828,828,910..."
4,5,"55983715,154954,164509,215627,1156970,15978631...","31,279,279,279,361,460,527,910,1056,1056,1343,..."


In [3]:
combined_entity_df = pd.read_csv("test_data/combined_entity.csv")

In [4]:
combined_text_df = pd.read_csv("test_data/combined_text.csv")

In [5]:
item_df = pd.read_csv("kensho-derived-wikimedia-data/items_filtered.csv")
item_aliases_df = pd.read_csv("kensho-derived-wikimedia-data/item_aliases_filtered.csv")
page_df = pd.read_csv("kensho-derived-wikimedia-data/page.csv")

In [6]:
data_array = item_df.to_numpy()
data_alias_array = item_aliases_df.to_numpy()
page_array = page_df.to_numpy()

In [7]:
item_dict_df = pd.read_csv("kensho-derived-wikimedia-data/item_dict.csv", delimiter = ",")

In [8]:
item_names = list(item_df['en_label'])
alias_names = list(item_aliases_df['en_alias'])

In [9]:
item_dict = {k:[int(i) for i in v] for k, v in zip(item_dict_df.en_label, item_dict_df.item_id.str.split(','))}

In [11]:
text_id = 30
sentence = list(combined_text_df[combined_text_df['text_id'] == text_id]['text'])[0]
candidate_entities = list(combined_entity_df[combined_entity_df['text_id'] == text_id]['entity'])


print(sentence)

for candidate in candidate_entities:
    print(candidate)

In mathematics and statistics, the arithmetic mean (, stress on third syllable of "arithmetic"), or simply the mean or average when the context is clear, is the sum of a collection of numbers divided by the count of numbers in the collection. The collection is often a set of results of an experiment or an observational study, or frequently a set of results from a survey. The term "arithmetic mean" is preferred in some contexts in mathematics and statistics because it helps distinguish it from other means, such as the geometric mean and the harmonic mean. In addition to mathematics and statistics, the arithmetic mean is used frequently in many diverse fields such as economics, anthropology, and history, and it is used in almost every academic field to some extent. For example, per capita income is the arithmetic average income of a nation's population. While the arithmetic mean is often used to report central tendencies, it is not a robust statistic, meaning that it is greatly influence

In [12]:
# first spacy entity recognition
# neighbor entities select most possible entity using word embeddings, target entity word embedding
# choose target entity using word embeddings of selected neigbor entities

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [13]:
doc = nlp(sentence)
print([(X.text, X.label_) for X in doc.ents])

[('third', 'ORDINAL')]


In [14]:
name = 'mean'
item_ids = item_dict.get(name)
entities = []
page_array_mention = []
for item_id in item_ids:
    page_array_indices = np.where(page_array[:,1]==item_id)[0]
    view_array = page_array[list(page_array_indices)][0]
    entities.append(view_array[2])
    page_array_mention.append(view_array)
    print(view_array)

[612 19033 'Arithmetic mean' 11987]
[4378617 1445196 'Larry Micheaux' 216]
[29264375 2303661 'Mean (song)' 2051]
[19192 2796622 'Mean' 21369]
[8675943 3303784 'Mean (album)' 256]
[15150843 6803558 'Mean (magazine)' 21]
[42048044 16977938 'MEAN (software bundle)' 11485]


In [17]:
# word embeddings of the neighbor entities
from gensim.sklearn_api import W2VTransformer
from gensim.models import Word2Vec
from wikipedia2vec import Wikipedia2Vec
import re
import gensim

# context is the whole vocabulary (use common_texts)
# wiki2vec = Wikipedia2Vec.load("model/enwiki_20180420_100d.pkl")
google2vec = gensim.models.KeyedVectors.load_word2vec_format('model/GoogleNews-vectors-negative300.bin', binary=True)  


# context_embeddings = [wiki2vec.get_word_vector(X.text) 
#                                  for X in doc.ents]+[wiki2vec.get_word_vector(X.label_.lower()) for X in doc.ents]

context_embeddings = [google2vec.wv[X.text] for X in doc.ents]+[google2vec.wv[X.label_.lower()] for X in doc.ents]


/Users/weiruchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [18]:
import scipy

distances = []
for entity in entities:
#     entity_embedding = wiki2vec.get_entity_vector(entity)
    
    entity_processed = re.split(r'\W+', re.sub(r'^\W+|\W+$', '', entity))
    entity_embedding = np.mean([google2vec.wv[w] for w in entity_processed if w in google2vec.wv], axis=0)
    
    distance = 0
    for context_embedding in context_embeddings:
        distance += scipy.spatial.distance.cosine(entity_embedding, context_embedding)
    distances.append(distance/len(context_embeddings))
min_index = distances.index(min(distances))
entity_selected = entities[min_index]
page_id, item_id = page_array_mention[min_index][0], page_array_mention[min_index][1]

page_df[page_df['item_id'] == item_id]

/Users/weiruchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


,page_id,item_id,title,views
30,612,19033,Arithmetic mean,11987


In [19]:
items_in_dict = list(item_dict.keys())

In [20]:
import random

def matching_entity(text_id):
    # get sentence
    sentence = list(combined_text_df[combined_text_df['text_id'] == text_id]['text'])[0]
    
    # get mentions from the sentence
    actual_pre = combined_entity_df[combined_entity_df['text_id'] == text_id]
    candidate_mentions = list(actual_pre['entity'])
    mention = ""
    for cand in candidate_mentions:
        if cand in items_in_dict:
            mention = cand
            break
    
    # get named entity recognitions
    doc = nlp(sentence)
    
    if mention == "":
        return None
    
    # get candidate entities
    item_ids = item_dict.get(mention)
    entities = []
    page_array_mention = []
    for item_id in item_ids:
        page_array_indices = np.where(page_array[:,1]==item_id)[0]
        view_array = page_array[list(page_array_indices)][0]
        entities.append(view_array[2])
        page_array_mention.append(view_array)
    
    # get word embeddings
    context = re.sub(r'^\W+|\W+$', '', sentence)
    context = re.split(r'\W+', context)
    context_embeddings = [np.mean([google2vec.wv[w] for w in X.text.split() if w in google2vec.wv]
                            , axis=0) for X in doc.ents]
    
    # get predicted entity
    distances = []
    for entity in entities:
#         entity_embedding = wiki2vec.get_entity_vector(entity)
        entity_processed = re.split(r'\W+', re.sub(r'^\W+|\W+$', '', entity))
        entity_embedding = np.mean([google2vec.wv[w] for w in entity_processed if w in google2vec.wv], axis=0)
    
        distance = 0
        for context_embedding in context_embeddings:
            distance += scipy.spatial.distance.cosine(entity_embedding, context_embedding)
        if len(context_embeddings) != 0:
            distances.append(distance/len(context_embeddings))
    if len(distances) != 0:
        min_index = distances.index(min(distances))
        entity_selected = entities[min_index]
        page_id, item_id = page_array_mention[min_index][0], page_array_mention[min_index][1]
    else:
        return None
    
    # get actual entity
    actual = list(actual_pre[actual_pre['entity'] == mention]['page_id'])[0]
    
    if page_id == actual:
        return 1
    else:
        return 0
    

In [21]:
correct = 0
count = 0
for i in range(20000):
    if i%20 == 0:
        print(i)
    if matching_entity(i) is not None:
        count += 1
        correct += matching_entity(i)

print("Accuracy rate is:", correct/count)

0


/Users/weiruchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/Users/weiruchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/Users/weiruchen/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/weiruchen/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020
4040
4060
4080
4100
4120
4140
4160
4180
4200
422